## Concat/Append, Merge and Groupby

**Concat**

In [1]:
import pandas as pd

Concatenate two series by row

In [2]:
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
pd.concat([s1, s2])

0    a
1    b
0    c
1    d
dtype: object

Concatenate two dataframes by row

In [3]:
df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 3], ['d', 4]],
                   columns=['letter', 'number'])
pd.concat([df1,df2])

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


While concatenating dataframes with non-overlappling columns, the columns outside the intersection will be filled with `NaN` values

In [4]:
df3 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']],
                   columns=['letter', 'number', 'animal'])
pd.concat([df1,df3],sort=False)

,letter,number,animal
0,a,1,NaN
1,b,2,NaN
0,c,3,cat
1,d,4,dog


In order to combine dataframes with only overlapping columns by passing `inner` to the `join` keyword

In [5]:
pd.concat([df1, df3], join="inner")

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


Combining dataframes horiozontally along the x-axis by passing in `axis=1`

In [6]:
df4 = pd.DataFrame([['bird', 'polly'], ['tiger', 'george']],
                   columns=['animal', 'name'])
pd.concat([df1, df4], axis=1)

,letter,number,animal,name
0,a,1,bird,polly
1,b,2,tiger,george


**Append**

Appending two dataframes is equivalent to concatenating two dataframes along the default axis = 0.

![pd_pivot](https://miro.medium.com/max/2000/1*NLnoAF5uOSBC2Y7IuzfM_Q.png)

In [7]:
df1.append(df2)

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


**Merge**

Pandas `merge()` function allows us to combine data objects bases on one or more keys in a similar way to a relational database.

When you use merge(), you’ll provide two required arguments:

- The left DataFrame
- The right DataFrame
After that, you can provide a number of optional arguments to define how your datasets are merged:

`how`: This defines what kind of merge to make. It defaults to 'inner', but other possible options include 'outer', 'left', and 'right'.

`on`: Use this to tell merge() which columns or indices (also called key columns or key indices) you want to join on. This is optional. If it isn’t specified, and left_index and right_index (covered below) are False, then columns from the two DataFrames that share names will be used as join keys. If you use on, then the column or index you specify must be present in both objects.

`left_on` and `right_on`: Use either of these to specify a column or index that is present only in the left or right objects that you are merging. Both default to None.

`left_index` and `right_index`: Set these to True to use the index of the left or right objects to be merged. Both default to False.

`suffixes`: This is a tuple of strings to append to identical column names that are not merge keys. This allows you to keep track of the origins of columns with the same name.

Joining two dataframes on a common key

In [8]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [9]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


Joining the dataframes when the common key has a different column in the two dataframes

In [10]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
pd.merge(df1,df3, left_on="employee", right_on="name")

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


Joining the dataframes on an index

In [11]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


Joining dataframes by the type of join

1. inner join

In [12]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

In [13]:
pd.merge(df6, df7, how='inner', on='name')

,name,food,drink
0,Mary,bread,wine


2. outer join

The `indicator` parameter can be set to True for creating a column `_merge` with information on the source of each row.

In [14]:
pd.merge(df6, df7, how='outer', on='name',indicator=True)

,name,food,drink,_merge
0,Peter,fish,NaN,left_only
1,Paul,beans,NaN,left_only
2,Mary,bread,wine,both
3,Joseph,NaN,beer,right_only


3. left join

In [15]:
pd.merge(df6, df7, how='left', on='name')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


4. right join

In [16]:
pd.merge(df6, df7, how='right', on='name')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


Overlapping columns

In [17]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})

In [18]:
pd.merge(df8, df9, how='inner',on='name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Because the output would have two conflicting column names, the merge function automatically appends a suffix `_x` or `_y` to make the output columns unique.

In [19]:
pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


**Group by**

Often the data needs to be analyzed by some categories. Pandas `groupby` method is similar to the `GROUP BY` statement that splits the data into separate groups to perform computations for a particular analysis.

To demonstarte some of the analysis using `groupby`, we will using a dataset of historical members of Congress.

In [20]:
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "https://theunitedstates.io/congress-legislators/legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12050 entries, 0 to 12049
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   last_name   12050 non-null  object        
 1   first_name  12050 non-null  category      
 2   birthday    11500 non-null  datetime64[ns]
 3   gender      12050 non-null  category      
 4   type        12050 non-null  category      
 5   state       12050 non-null  category      
 6   party       11818 non-null  category      
dtypes: category(5), datetime64[ns](1), object(1)
memory usage: 317.3+ KB


,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN


Let's try to get the count of Congressional members by state. In SQL, we can do this by using the below query:

`SELECT state, count(name)
FROM df
GROUP BY state
ORDER BY state;`

In [21]:
df.groupby('state')['first_name'].count().head()

state
AK     16
AL    209
AR    117
AS      2
AZ     49
Name: first_name, dtype: int64

Note: In the above example, SQL query has an `ORDER BY` clause whereas `.groupby()` does not. That’s because `.groupby()` does this by default through its parameter sort, which is True unless you tell it otherwise

Next, let's see an example of grouping jointly on two columns that finds the count of Congressional members broken out by state and gender. 

`SELECT state, gender, count(name)
FROM df
GROUP BY state, gender
ORDER BY state, gender;`

In [22]:
df.groupby(['state','gender'])['last_name'].count()

state  gender
AK     F           0
       M          16
AL     F           4
       M         205
AR     F           5
                ... 
WI     M         198
WV     F           1
       M         119
WY     F           1
       M          39
Name: last_name, Length: 116, dtype: int64

We can use `ngroups` attribute to get the number of group

In [23]:
df.groupby('gender').ngroups

2

Next, we will read a dummy sales data to deep dive into pandas `groupby` method and demonstrate some advanced examples

In [24]:
df = pd.read_excel("https://pbpython.com/extras/sales-funnel.xlsx")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Account   17 non-null     int64 
 1   Name      17 non-null     object
 2   Rep       17 non-null     object
 3   Manager   17 non-null     object
 4   Product   17 non-null     object
 5   Quantity  17 non-null     int64 
 6   Price     17 non-null     int64 
 7   Status    17 non-null     object
dtypes: int64(3), object(5)
memory usage: 1.2+ KB


,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [25]:
df

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
5,218895,Kulas Inc,Daniel Hilton,Debra Henley,CPU,2,40000,pending
6,218895,Kulas Inc,Daniel Hilton,Debra Henley,Software,1,10000,presented
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won


Generally speaking, `group by` is referring to a process involving one or more of the following steps:

1. Splitting the data into groups. 
2. Applying a function to each group independently,
3. Combining the results into a data structure.

The .groups attribute will give you a dictionary of {group name: group label} pairs.

In [26]:
df.groupby('Rep').groups

{'Cedric Moss': [9, 10, 11, 12], 'Craig Booker': [0, 1, 2, 3], 'Daniel Hilton': [4, 5, 6], 'John Smith': [7, 8], 'Wendy Yule': [13, 14, 15, 16]}

Each value in the above output is a sequence of the index locations for the rows belonging to that particular group.

In [27]:
df.groupby('Rep').groups['John Smith']

Int64Index([7, 8], dtype='int64')

You can also use .get_group() as a way to drill down to the sub-table from a single group

In [28]:
df.groupby('Rep').get_group('John Smith')

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined


Now, lets try to show this process by an example. Suppose we want to see the following information for each sales rep and company:

- Maximum and minimum price of the orders
- Count of orders

In [29]:
df.groupby(['Rep','Name']).Price.agg(['max','min','count'])

max     min  count
Rep           Name                                               
Cedric Moss   Herman LLC                     65000   65000      1
              Purdy-Kunde                    30000   30000      1
              Stokes LLC                     10000    5000      2
Craig Booker  Fritsch, Russel and Anderson   35000   35000      1
              Trantow-Barrows                30000    5000      3
Daniel Hilton Kiehn-Spinka                   65000   65000      1
              Kulas Inc                      40000   10000      2
John Smith    Barton LLC                     35000   35000      1
              Jerde-Hilpert                   5000    5000      1
Wendy Yule    Kassulke, Ondricka and Metz     7000    7000      1
              Keeling LLC                   100000  100000      1
              Koepp Ltd                      65000    5000      2